In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
load_dotenv()
DB_URL = os.getenv("DB_URL")
engine = create_engine(DB_URL)
# select 26 rows from SQL table to insert in dataframe.
query = """
select c.ContactPersoonId, i.CampagneId, i.CampagneNaam, a.Ondernemingsaard, a.Ondernemingstype, a.PrimaireActiviteit, f.Naam as Functie
from Contactfiche c
join Account a on a.AccountId = c.AccountId
join Inschrijving i on i.ContactficheId = c.ContactPersoonId
join ContactficheFunctie cf on cf.ContactpersoonId = c.ContactPersoonId
join Functie f on f.FunctieId = cf.FunctieId
where CampagneId is not null;
"""
df = pd.read_sql(query, engine)
df.set_index('ContactPersoonId', inplace=True)
df["rating"] = 1

df.head()

,CampagneId,CampagneNaam,Ondernemingsaard,Ondernemingstype,PrimaireActiviteit,Functie,rating
ContactPersoonId,,,,,,,
D9303EA2-57E5-EB11-8121-001DD8B72B61,F090DEF9-2A5A-EB11-811A-001DD8B72B62,OV - Kick-Off Community Bouw en Vastgoed,Diensten,Familiebedrijf,Vastgoed,Medewerker,1
451DF235-4B73-E111-B43A-00505680000A,ED956944-5F6F-EA11-8110-001DD8B72B62,OV-Corona Round Tables - Sales,Diensten,Bedrijf,Consultancy,Bedrijfsleider,1
4235A499-B670-ED11-9561-6045BD895CDC,604B37A4-C064-ED11-9561-6045BD895B5A,OV-MATCH-Welkom@Voka- 28 februari 2023,Diensten,Bedrijf,Milieu,Verantwoordelijke Commercieel,1
8A72E0E1-4C0A-E811-80EF-001DD8B72B61,79DCACF5-146D-E711-80E8-001DD8B72B61,OV-Nieuwe klanten via LinkedIn,Productie & Diensten,Bedrijf,Overige industrie & diensten,Verantwoordelijke Marketing,1
FF17ED97-0942-E611-80D6-005056B06EC4,96ACAD9A-E7E3-EB11-8124-001DD8B72B62,OV-Start2Export2 Mexico traject,Diensten,Bedrijf,Voeding,Bedrijfsleider,1


In [3]:
df_pivot = pd.pivot_table(df, index='ContactPersoonId', columns=['Ondernemingsaard', 'Ondernemingstype', 'PrimaireActiviteit', 'Functie'], values='rating', fill_value = 0)
df_pivot
# df_pivot
# df_pivot[df_pivot[' OV-JO-Breakfastclub December 2019'] == 1]
# df_contacts_features = df[['Ondernemingsaard', 'Ondernemingstype', 'PrimaireActiviteit']]
# df_contacts_features = pd.get_dummies(df_contacts_features)
# df_contacts_features

Ondernemingsaard                                      Diensten             \
Ondernemingstype                                       Bedrijf              
PrimaireActiviteit                   Accountancy & boekhouding              
Functie                                         Bedrijfsleider Bestuurder   
ContactPersoonId                                                            
00169619-E322-E911-80FB-001DD8B72B62                       0.0        0.0   
0017416A-2C6E-E111-B43A-00505680000A                       0.0        0.0   
0019C15A-6481-E611-80DE-001DD8B72B61                       0.0        0.0   
00223C8E-467F-E311-BBFD-005056B06EB4                       0.0        0.0   
00231824-53EA-ED11-8849-6045BD895420                       0.0        0.0   
...                                                        ...        ...   
FFF0C643-DAE4-ED11-A7C7-000D3A4AB78E                       0.0        0.0   
FFF68074-EB93-E911-80FF-001DD8B72B62                       0.0        0.0   
FFF68536-5DE0-E111-8A53-984BE17C2819                       0.0        0.0   
FFFAE2B6-11D5-EC11-A7B5-000D3ABD1F85                       0.0        0.0   
FFFEA9CB-ED93-EC11-B400-000D3A2B10EB                       0.0        0.0   

Ondernemingsaard                                           \
Ondernemingstype                                            
PrimaireActiviteit                                          
Functie                              Contact Lidmaatschap   
ContactPersoonId                                            
00169619-E322-E911-80FB-001DD8B72B62                  0.0   
0017416A-2C6E-E111-B43A-00505680000A                  0.0   
0019C15A-6481-E611-80DE-001DD8B72B61                  0.0   
00223C8E-467F-E311-BBFD-005056B06EB4                  0.0   
00231824-53EA-ED11-8849-6045BD895420                  0.0   
...                                                   ...   
FFF0C643-DAE4-ED11-A7C7-000D3A4AB78E                  0.0   
FFF68074-EB93-E911-80FF-001DD8B72B62                  0.0   
FFF68536-5DE0-E111-8A53-984BE17C2819                  0.0   
FFFAE2B6-11D5-EC11-A7B5-000D3ABD1F85                  0.0   
FFFEA9CB-ED93-EC11-B400-000D3A2B10EB                  0.0   

Ondernemingsaard                                           \
Ondernemingstype                                            
PrimaireActiviteit                                          
Functie                              Directie of kaderlid   
ContactPersoonId                                            
00169619-E322-E911-80FB-001DD8B72B62                  0.0   
0017416A-2C6E-E111-B43A-00505680000A                  0.0   
0019C15A-6481-E611-80DE-001DD8B72B61                  0.0   
00223C8E-467F-E311-BBFD-005056B06EB4                  0.0   
00231824-53EA-ED11-8849-6045BD895420                  0.0   
...                                                   ...   
FFF0C643-DAE4-ED11-A7C7-000D3A4AB78E                  0.0   
FFF68074-EB93-E911-80FF-001DD8B72B62                  0.0   
FFF68536-5DE0-E111-8A53-984BE17C2819                  0.0   
FFFAE2B6-11D5-EC11-A7B5-000D3ABD1F85                  0.0   
FFFEA9CB-ED93-EC11-B400-000D3A2B10EB                  0.0   

Ondernemingsaard                                                      \
Ondernemingstype                                                       
PrimaireActiviteit                                                     
Functie                              Management Assistent Medewerker   
ContactPersoonId                                                       
00169619-E322-E911-80FB-001DD8B72B62                  0.0        0.0   
0017416A-2C6E-E111-B43A-00505680000A                  0.0        0.0   
0019C15A-6481-E611-80DE-001DD8B72B61                  0.0        0.0   
00223C8E-467F-E311-BBFD-005056B06EB4                  0.0        0.0   
00231824-53EA-ED11-8849-6045BD895420                  0.0        0.0   
...                                                   ...        ...   
FFF0C643-DAE4-ED11-A7

In [13]:
similarity_matrix = cosine_similarity(df_pivot)
similarity_matrix_df = pd.DataFrame(similarity_matrix, index=df_pivot.index, columns=df_pivot.index)
del similarity_matrix
# similarity_matrix_df


KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'ContactPersoonId' not found in columns."

In [12]:
similarity_matrix_df

ContactPersoonId,00169619-E322-E911-80FB-001DD8B72B62,0017416A-2C6E-E111-B43A-00505680000A,0019C15A-6481-E611-80DE-001DD8B72B61,00223C8E-467F-E311-BBFD-005056B06EB4,00231824-53EA-ED11-8849-6045BD895420,0025D44A-C19F-E311-B1AE-005056B06EC4,0033DB6B-815B-E511-8718-005056B06EC4,0036CCCE-B36F-E111-B43A-00505680000A,003740F4-F4A0-EB11-811E-001DD8B72B62,00426A48-F851-EC11-8C62-000D3ABFC672,...,FFC4A1F5-7BA0-EC11-B400-6045BD93C7AE,FFCF2AB4-BA72-E111-B43A-00505680000A,FFD3449C-3A6C-E111-B43A-00505680000A,FFD90821-3A6F-E111-B43A-00505680000A,FFE9091D-AC4D-E211-9B51-005056B06EC4,FFF0C643-DAE4-ED11-A7C7-000D3A4AB78E,FFF68074-EB93-E911-80FF-001DD8B72B62,FFF68536-5DE0-E111-8A53-984BE17C2819,FFFAE2B6-11D5-EC11-A7B5-000D3ABD1F85,FFFEA9CB-ED93-EC11-B400-000D3A2B10EB
ContactPersoonId,,,,,,,,,,,,,,,,,,,,,
00169619-E322-E911-80FB-001DD8B72B62,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0017416A-2C6E-E111-B43A-00505680000A,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0019C15A-6481-E611-80DE-001DD8B72B61,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00223C8E-467F-E311-BBFD-005056B06EB4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00231824-53EA-ED11-8849-6045BD895420,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FFF0C643-DAE4-ED11-A7C7-000D3A4AB78E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
FFF68074-EB93-E911-80FF-001DD8B72B62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
FFF68536-5DE0-E111-8A53-984BE17C2819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [5]:
select_contact = "00169619-E322-E911-80FB-001DD8B72B62"
similarities = similarity_matrix_df[select_contact].drop(select_contact)
weights = similarities/similarities.sum()

In [6]:
#num of silimar users
n = 10
#set a threshold for similarity
user_similarity_threshold =  0.5
# Get top n similar users
similar_users = similarity_matrix_df[similarity_matrix_df[select_contact]>user_similarity_threshold][select_contact].sort_values(ascending=False)[:n]

In [7]:
similar_users

ContactPersoonId
00169619-E322-E911-80FB-001DD8B72B62    1.0
29631920-B16A-E111-B43A-00505680000A    1.0
F1862259-775F-E911-80FD-001DD8B72B62    1.0
B9AD6B5C-0473-E111-B43A-00505680000A    1.0
892C77B4-0992-E611-80DE-001DD8B72B61    1.0
01218324-0942-E611-80D6-005056B06EC4    1.0
6BEDA1C3-C08D-ED11-81AD-6045BD895D85    1.0
298B17CE-4358-E311-BBFD-005056B06EB4    1.0
6E9E2D4E-A88E-EA11-810F-001DD8B72B61    1.0
0D4A271E-A66B-E111-B43A-00505680000A    1.0
Name: 00169619-E322-E911-80FB-001DD8B72B62, dtype: float64

In [8]:
done_campaigns = df.loc[df.index == select_contact]['CampagneId']
done_campaigns

ContactPersoonId
00169619-E322-E911-80FB-001DD8B72B62    67B929FF-34F2-E811-80F9-001DD8B72B61
00169619-E322-E911-80FB-001DD8B72B62    EA3C6FE0-758E-E811-80F3-001DD8B72B61
00169619-E322-E911-80FB-001DD8B72B62    EA3C6FE0-758E-E811-80F3-001DD8B72B61
00169619-E322-E911-80FB-001DD8B72B62    67B929FF-34F2-E811-80F9-001DD8B72B61
00169619-E322-E911-80FB-001DD8B72B62    67B929FF-34F2-E811-80F9-001DD8B72B61
00169619-E322-E911-80FB-001DD8B72B62    EA3C6FE0-758E-E811-80F3-001DD8B72B61
00169619-E322-E911-80FB-001DD8B72B62    EA3C6FE0-758E-E811-80F3-001DD8B72B61
00169619-E322-E911-80FB-001DD8B72B62    67B929FF-34F2-E811-80F9-001DD8B72B61
Name: CampagneId, dtype: object

In [9]:
# Movies that similar users watched.
similar_campagnes = df[df.index.isin(similar_users.index)].replace(0, np.nan).dropna(axis=1, how='all')

#similar_user_movies = movies_ratings_pivot[movies_ratings_pivot.index.isin(top similar_users.index)].dropna(axis=1, how='all')
similar_campagnes = similar_campagnes.drop(select_contact, axis=0)
similar_campagnes

,CampagneId,CampagneNaam,Ondernemingsaard,Ondernemingstype,PrimaireActiviteit,Functie,rating
ContactPersoonId,,,,,,,
01218324-0942-E611-80D6-005056B06EC4,AF18B713-6DB5-E711-80EC-001DD8B72B62,OV-NW-Voka connect Aalst 2018,Productie & Diensten,Bedrijf,Technologische industrie & diensten,Bedrijfsleider,1
0D4A271E-A66B-E111-B43A-00505680000A,46C0357E-13E0-ED11-A7C6-6045BD895FE4,OV-NW/BB-Voka Politica Erpe-Mere 2023,Productie & Diensten,Bedrijf,Technologische industrie & diensten,Bedrijfsleider,1
298B17CE-4358-E311-BBFD-005056B06EB4,2DAA64E1-273E-E811-80EF-001DD8B72B62,OV-P-PLATO 2018- After summer drink,Productie & Diensten,Bedrijf,Technologische industrie & diensten,Contact Lidmaatschap,1
892C77B4-0992-E611-80DE-001DD8B72B61,99C6798B-BE7E-E811-80F3-001DD8B72B61,OV-Praktisch balans lezen,Productie & Diensten,Bedrijf,Technologische industrie & diensten,Bedrijfsleider,1
01218324-0942-E611-80D6-005056B06EC4,AF18B713-6DB5-E711-80EC-001DD8B72B62,OV-NW-Voka connect Aalst 2018,Productie & Diensten,Bedrijf,Technologische industrie & diensten,Bedrijfsleider,1
...,...,...,...,...,...,...,...
6E9E2D4E-A88E-EA11-810F-001DD8B72B61,0F57FD1E-6FA6-EA11-8112-001DD8B72B62,OV-NW-Netwekkers Digitaal met WVL-1 juli 2020,Productie & Diensten,Bedrijf,Technologische industrie & diensten,Contact Lidmaatschap,1
6E9E2D4E-A88E-EA11-810F-001DD8B72B61,241B4E80-D6A0-EA11-8110-001DD8B72B61,OV-JO-Inspiratiesessie Growth Marketing,Productie & Diensten,Bedrijf,Technologische industrie & diensten,Contact Lidmaatschap,1
6E9E2D4E-A88E-EA11-810F-001DD8B72B61,E0C3ABFC-9CBB-EA11-8113-001DD8B72B62,OV-JO Breakfastclub November 2020,Productie & Diensten,Bedrijf,Technologische industrie & diensten,Contact Lidmaatschap,1


In [10]:
similar_campagnes[['CampagneId', 'CampagneNaam']]


,CampagneId,CampagneNaam
ContactPersoonId,,
01218324-0942-E611-80D6-005056B06EC4,AF18B713-6DB5-E711-80EC-001DD8B72B62,OV-NW-Voka connect Aalst 2018
0D4A271E-A66B-E111-B43A-00505680000A,46C0357E-13E0-ED11-A7C6-6045BD895FE4,OV-NW/BB-Voka Politica Erpe-Mere 2023
298B17CE-4358-E311-BBFD-005056B06EB4,2DAA64E1-273E-E811-80EF-001DD8B72B62,OV-P-PLATO 2018- After summer drink
892C77B4-0992-E611-80DE-001DD8B72B61,99C6798B-BE7E-E811-80F3-001DD8B72B61,OV-Praktisch balans lezen
01218324-0942-E611-80D6-005056B06EC4,AF18B713-6DB5-E711-80EC-001DD8B72B62,OV-NW-Voka connect Aalst 2018
...,...,...
6E9E2D4E-A88E-EA11-810F-001DD8B72B61,0F57FD1E-6FA6-EA11-8112-001DD8B72B62,OV-NW-Netwekkers Digitaal met WVL-1 juli 2020
6E9E2D4E-A88E-EA11-810F-001DD8B72B61,241B4E80-D6A0-EA11-8110-001DD8B72B61,OV-JO-Inspiratiesessie Growth Marketing
6E9E2D4E-A88E-EA11-810F-001DD8B72B61,E0C3ABFC-9CBB-EA11-8113-001DD8B72B62,OV-JO Breakfastclub November 2020


In [11]:
similar_campagnes_not_done = similar_campagnes[~similar_campagnes['CampagneId'].isin(done_campaigns)][['CampagneId', 'CampagneNaam']]
similar_campagnes_not_done.set_index('CampagneId', inplace=True)
similar_campagnes_not_done

,CampagneNaam
CampagneId,
AF18B713-6DB5-E711-80EC-001DD8B72B62,OV-NW-Voka connect Aalst 2018
46C0357E-13E0-ED11-A7C6-6045BD895FE4,OV-NW/BB-Voka Politica Erpe-Mere 2023
2DAA64E1-273E-E811-80EF-001DD8B72B62,OV-P-PLATO 2018- After summer drink
99C6798B-BE7E-E811-80F3-001DD8B72B61,OV-Praktisch balans lezen
AF18B713-6DB5-E711-80EC-001DD8B72B62,OV-NW-Voka connect Aalst 2018
...,...
0F57FD1E-6FA6-EA11-8112-001DD8B72B62,OV-NW-Netwekkers Digitaal met WVL-1 juli 2020
241B4E80-D6A0-EA11-8110-001DD8B72B61,OV-JO-Inspiratiesessie Growth Marketing
E0C3ABFC-9CBB-EA11-8113-001DD8B72B62,OV-JO Breakfastclub November 2020
